In [53]:
import nucleus
from nucleus import Scene, LidarScene, SceneDatasetItem, NucleusClient

In [54]:
BUCKET = "scale-ml"
S3_BUCKET = "s3://scale-ml/"
OBJECT_PATH = "pandaset/023/"
SCENE_URL = "s3://scale-ml/pandaset/023/"

IMAGE_TYPE = "image"
POINTCLOUD_TYPE = "pointcloud"

In [3]:
import sys
!{sys.executable} -m pip install boto3

You should consider upgrading via the '/Users/drewkaul/Library/Caches/pypoetry/virtualenvs/scale-nucleus-nqXix_D8-py3.8/bin/python -m pip install --upgrade pip' command.


In [4]:
import boto3

client = boto3.client('s3')
paginator = client.get_paginator('list_objects_v2')
result = paginator.paginate(Bucket=BUCKET, Prefix=OBJECT_PATH, StartAfter=OBJECT_PATH)

In [6]:
image_paths = []
pointcloud_paths = []

def is_image_path(object_path):
    return "camera/" in object_path and ".jpg" in object_path

def is_pointcloud_path(object_path):
    return "lidar/" in object_path and ".json" in object_path

for page in result:
    if "Contents" in page:
        for key in page["Contents"]:
            object_path = key["Key"]
            
            if is_image_path(object_path):
                image_paths.append(object_path)
            
            if is_pointcloud_path(object_path):
                pointcloud_paths.append(object_path)

In [10]:
print(len(image_paths))
print(image_paths[0])
# print(image_paths)

print(len(pointcloud_paths))
print(pointcloud_paths[0])
# print(pointcloud_paths)

480
pandaset/023/camera/back_camera/00.jpg
80
pandaset/023/lidar/00.json


In [67]:
import os
import re

scene_number = re.split('/', OBJECT_PATH)[1]
scene_ref_id = f"scene-{scene_number}"
scene = LidarScene(scene_ref_id)

for image_path in image_paths:
    image_url = os.path.join(S3_BUCKET, image_path)
    
    tokens = re.split('/|\.', image_path)
    frame_idx = int(tokens[-2])
    sensor_name = tokens[-3]
    reference_id = f"scene-{scene_number}-frame-{frame_idx}-{sensor_name}"
    
    item = SceneDatasetItem(image_url, IMAGE_TYPE, reference_id)
    scene.add_item(frame_idx, sensor_name, item)

for pointcloud_path in pointcloud_paths:
    pointcloud_url = os.path.join(S3_BUCKET, pointcloud_path)
    
    tokens = re.split('/|\.', pointcloud_path)
    frame_idx = int(tokens[-2])
    sensor_name = tokens[-3]
    reference_id = f"scene-{scene_number}-frame-{frame_idx}-{sensor_name}"
    
    item = SceneDatasetItem(pointcloud_url, POINTCLOUD_TYPE, reference_id)
    scene.add_item(frame_idx, sensor_name, item)
    

In [68]:
print("number of frames:", len(scene.frames_dict))
frame_0 = scene.frames_dict[0]
print("number of sensors per frame:", len(frame_0.items.values()))
print(frame_0)

number of frames: 80
number of sensors per frame: 7
Frame(items={'back_camera': SceneDatasetItem(url='s3://scale-ml/pandaset/023/camera/back_camera/00.jpg', type='image', reference_id='scene-023-frame-0-back_camera', metadata=None, camera_params=None), 'front_camera': SceneDatasetItem(url='s3://scale-ml/pandaset/023/camera/front_camera/00.jpg', type='image', reference_id='scene-023-frame-0-front_camera', metadata=None, camera_params=None), 'front_left_camera': SceneDatasetItem(url='s3://scale-ml/pandaset/023/camera/front_left_camera/00.jpg', type='image', reference_id='scene-023-frame-0-front_left_camera', metadata=None, camera_params=None), 'front_right_camera': SceneDatasetItem(url='s3://scale-ml/pandaset/023/camera/front_right_camera/00.jpg', type='image', reference_id='scene-023-frame-0-front_right_camera', metadata=None, camera_params=None), 'left_camera': SceneDatasetItem(url='s3://scale-ml/pandaset/023/camera/left_camera/00.jpg', type='image', reference_id='scene-023-frame-0-lef

In [55]:
API_KEY = "live_318209d04e3746dbafbe1f195a4a1872"
TEST_DATASET_NAME = "test_dataset_3d"

In [70]:
# new_pointcloud_url = "s3://scale-ml/pandaset/023/lidar_frame_0.json"
# for frame in scene.frames:
#     for sensor, item in frame.items.items():
#         if item.type == "pointcloud":
#             new_item = SceneDatasetItem(new_pointcloud_url, item.type, item.reference_id)
#             frame.items[sensor] = new_item

In [69]:
client = NucleusClient(API_KEY)
dataset = client.create_dataset(TEST_DATASET_NAME)

scenes = [scene]
response = dataset.append(scenes)
print(response)

{'dataset_id': 'ds_c4ack3wpfpd5q3w8rbn0', 'new_scenes': 0, 'ignored_scenes': 0, 'scenes_errored': 1, 'errors': ['Error: The resource at s3://scale-ml/pandaset/023/lidar/04.json could not be downloaded: Status code 403']}


In [ ]:
# read in annotations from local dir
# upload cuboid annotations
dataset.annotate(annotations)

In [ ]:
# fn to construct Frame
# call scene.add_frame
# call frame.add_item
